In [1]:
import numpy as np
import numpy.linalg as nl
import scipy
import scipy.linalg as sl
import matplotlib.pyplot as plt
import sys
sys.path.append('./src')

np.set_printoptions(suppress=True)

import pauli2qumode
import importlib
importlib.reload(pauli2qumode)
from pauli2qumode import *

In [5]:
# pauli_terms = ["IYY"]
# coefficients = [1]

# # Pauli matrices for constructing target operator.
# I = np.array([[1, 0], [0, 1]], dtype=complex)
# X = np.array([[0, 1], [1, 0]], dtype=complex)
# Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
# Z = np.array([[1, 0], [0, -1]], dtype=complex)
# pauli_map = {'I': I, 'X': X, 'Y': Y, 'Z': Z}

# W = 1
# for term, coeff in zip(pauli_terms, coefficients):
#     for p in term:
#         W = np.kron(pauli_map[p],W)
#     W = coeff * W

from lchs_pauli import pauli_arr_to_mat
use_circuit = True
hf = 8
A_pauli_arr = ['II', 'IX', 'XX', 'YY']
A_coef_arr = np.array([2, -1, -0.5, -0.5]) * 1/(hf**2)
L_pauli_arr = A_pauli_arr
L_coef_arr = A_coef_arr.copy()
H_pauli_arr = []
H_coef_arr = np.array([])
#
A = pauli_arr_to_mat(A_coef_arr, A_pauli_arr)

import numpy
nq = 2 #2
## Define random A and u0
dim = 2**nq
rng = numpy.random.Generator(numpy.random.PCG64(178984893489))


## Random numpy
from lchs import cart_decomp
# A = (numpy.matrix(rng.random((dim,dim)),dtype=complex)-0.5)  + 1j*(numpy.matrix(rng.random((dim,dim)),dtype=complex)-0.5)
# A = 0.5*A + 0.5*numpy.identity(dim)
L,H = cart_decomp(A)

N_t = 1
N_d = 5

res_dict = var_recd(A, N_t=N_t, N_d=N_d, exp_coeff=-1j*0.5, use_circuit=use_circuit)
res_dict

[[ 0.99984742-0.01562389j  0.00012206+0.00781139j -0.00003051+0.00000048j
  -0.        -0.00000008j]
 [ 0.00012206+0.00781139j  0.99981691-0.01562341j  0.00012206+0.00781131j
  -0.00003051+0.00000048j]
 [-0.00003051+0.00000048j  0.00012206+0.00781131j  0.99981691-0.01562341j
   0.00012206+0.00781139j]
 [-0.        -0.00000008j -0.00003051+0.00000048j  0.00012206+0.00781139j
   0.99984742-0.01562389j]]
Optimization terminated successfully.
         Current function value: 0.000008
         Iterations: 159
         Function evaluations: 3784
         Gradient evaluations: 172


{'W': array([[ 0.99984742-0.01562389j,  0.00012206+0.00781139j,
         -0.00003051+0.00000048j, -0.        -0.00000008j],
        [ 0.00012206+0.00781139j,  0.99981691-0.01562341j,
          0.00012206+0.00781131j, -0.00003051+0.00000048j],
        [-0.00003051+0.00000048j,  0.00012206+0.00781131j,
          0.99981691-0.01562341j,  0.00012206+0.00781139j],
        [-0.        -0.00000008j, -0.00003051+0.00000048j,
          0.00012206+0.00781139j,  0.99984742-0.01562389j]]),
 'fun': 7.702325695033441e-06,
 'full_params': array([ 1.        ,  3.20100697,  0.22395412, -0.08428219,  0.99247415,
         3.19512068,  3.37015354,  3.08073384, -0.69337357, -0.00773862,
         5.21764389,  3.29816556, -0.49915409, -2.84373666,  1.5084845 ,
         0.10168551, -2.66053356,  2.84586607,  1.50357084,  3.19978305,
         3.50491643]),
 'lambdas': array([1.]),
 'betas_mag': array([[ 3.20100697,  3.19512068, -0.00773862, -2.84373666,  2.84586607]]),
 'betas_ang': array([[0.22395412, 3.37015

In [ ]:
res_dict['circuit'].draw()

## Compare gates in the paper and in the Bosonic Qiskit

In [ ]:
import c2qa
from c2qa.operators import CVOperators

L = 2
beta = 2 ## use the manner

Xgate = np.array([[0,1],[1,0]])
Id = np.identity(L)
XI = np.kron(Xgate, Id)

ECDbq = CVOperators().ecd(beta/2, L).todense()
XI.dot(ECDbq)

In [ ]:
b = np.zeros((L, L), dtype=complex)
for m in range(1, L):
    b[m - 1, m] = np.sqrt(m)
b_dag = b.T

ecd_gate(beta, b, b_dag).real

In [ ]:
theta = 0.5
phi = 0.2

import qiskit
rz1 = qiskit.circuit.library.RZGate(phi).to_matrix()
rx = qiskit.circuit.library.RXGate(theta).to_matrix()
rz2 = qiskit.circuit.library.RZGate(-phi).to_matrix()

rz1 @ rx @ rz2

In [ ]:
rotation_gate(theta, phi)

In [ ]:
import qiskit.quantum_info
from utils_synth import qiskit_normal_order_switch


L = 2
beta_mag = 2
beta_ang = 1
theta = 0.5
phi = 0.2

beta = beta_mag * np.exp(1j * beta_ang)
param_test = [beta_mag, beta_ang, theta, phi]

qr = qiskit.QuantumRegister(size=1)
qmr = c2qa.QumodeRegister(num_qumodes=1, num_qubits_per_qumode=int( np.log2(L) ))
circuit = c2qa.CVCircuit(qr, qmr)

## Rotation
circuit.rz(-phi, qr[0])
circuit.rx(theta, qr[0])
circuit.rz(phi, qr[0])
## ECD
circuit.cv_ecd(beta/2, qmr[0], qr[0], duration=100, unit="ns")
circuit.x(qr)

print(circuit.draw())
qiskit_normal_order_switch( qiskit.quantum_info.Operator(circuit).to_matrix() )

In [ ]:
b = np.zeros((L, L), dtype=complex)
for m in range(1, L):
    b[m - 1, m] = np.sqrt(m)
b_dag = b.T

simulate_unitary_sequence([param_test], 1, b, b_dag, L)